# Admin Dashboard & Analytics Module

All admin dashboard routes, stats queries, analytics, and ZIP coverage logic migrated from `app.py`.

In [ ]:
# --- Admin dashboard, analytics, and settings ---
from flask import render_template, flash, redirect, url_for, session
from contextlib import closing
import datetime

# Import get_db from the db module
import sys
from db import get_db

def get_admin_dashboard(request):
    """
    Main function called by app.py for the /admin route
    """
    with closing(get_db()) as db:
        cur = db.cursor()
        
        # Get counts
        cur.execute("SELECT COUNT(*) FROM services WHERE active = 1")
        services_count = cur.fetchone()[0]
        
        cur.execute("SELECT COUNT(*) FROM products WHERE active = 1")
        products_count = cur.fetchone()[0]
        
        cur.execute("SELECT COUNT(*) FROM zips")
        zips_count = cur.fetchone()[0]
        
        cur.execute("SELECT COUNT(*) FROM leads")
        leads_count = cur.fetchone()[0]
        
        cur.execute("SELECT COUNT(*) FROM providers WHERE active = 1")
        providers_count = cur.fetchone()[0]
        
        # Get ALL services with provider information for the table
        cur.execute("""
            SELECT s.id, s.title, s.description, s.price, s.active, s.created_at, s.provider_id,
                   COALESCE(p.business_name, p.first_name, s.posted_by, 'Admin') as provider_name
            FROM services s
            LEFT JOIN providers p ON s.provider_id = p.id
            ORDER BY s.created_at DESC
        """)
        all_services = cur.fetchall()
        
        # Get recent leads
        cur.execute("""
            SELECT l.id, l.name, l.email, l.service, l.created_at, p.business_name as provider_name
            FROM leads l
            LEFT JOIN providers p ON l.provider_id = p.id
            ORDER BY l.created_at DESC
            LIMIT 10
        """)
        recent_leads = cur.fetchall()
        
        # Get ZIP codes with coverage
        cur.execute("SELECT zip, radius_miles FROM zips ORDER BY zip")
        zips = cur.fetchall()
        
        # Get profile for template
        cur.execute("SELECT first_name, business_name FROM profile WHERE id = 1")
        profile_row = cur.fetchone()
        profile = {
            "first_name": profile_row["first_name"] if profile_row else "",
            "business_name": profile_row["business_name"] if profile_row else "Your Service Provider"
        }
    
    return render_template(
        "admin_dashboard.html",
        services_count=services_count,
        products_count=products_count,
        zips_count=zips_count,
        leads_count=leads_count,
        providers_count=providers_count,
        all_services=all_services,
        recent_leads=recent_leads,
        zips=zips,
        profile=profile,
        title="Admin Dashboard"
    )

In [ ]:
def get_admin_leads(request):
    """
    Main function called by app.py for the /admin/leads route
    """
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute("""
            SELECT l.id, l.name, l.email, l.phone, l.zip, l.address, l.service, l.message, 
                   l.status, l.created_at, l.provider_id,
                   COALESCE(p.business_name, p.first_name, 'Unassigned') as provider_name
            FROM leads l
            LEFT JOIN providers p ON l.provider_id = p.id
            ORDER BY l.created_at DESC
        """)
        leads = cur.fetchall()
        
        # Get providers for assignment dropdown
        cur.execute("SELECT id, business_name, first_name FROM providers WHERE active = 1 ORDER BY business_name")
        providers = cur.fetchall()
    
    return render_template("admin_leads.html", leads=leads, providers=providers, title="Manage Leads")

def get_admin_assign_leads(request):
    """
    Main function called by app.py for the /admin/assign-leads route
    """
    with closing(get_db()) as db:
        cur = db.cursor()
        
        # Get unassigned leads
        cur.execute("""
            SELECT l.id, l.name, l.email, l.phone, l.zip, l.service, l.created_at
            FROM leads l
            WHERE l.provider_id = 0 OR l.provider_id IS NULL
            ORDER BY l.created_at DESC
        """)
        unassigned_leads = cur.fetchall()
        
        # Get all active providers
        cur.execute("SELECT id, business_name, first_name, base_zip FROM providers WHERE active = 1 ORDER BY business_name")
        providers = cur.fetchall()
    
    return render_template("admin_assign_leads.html", 
                         unassigned_leads=unassigned_leads, 
                         providers=providers, 
                         title="Assign Leads to Providers")

def get_admin_events(request):
    """
    Main function called by app.py for the /admin/events route
    """
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute("""
            SELECT e.id, e.title, e.description, e.date, e.location, e.zip, e.created_at,
                   p.business_name, p.first_name
            FROM events e
            LEFT JOIN providers p ON e.provider_id = p.id
            ORDER BY e.date DESC
        """)
        events = cur.fetchall()
    
    return render_template("admin_events.html", events=events, title="Manage Provider Events")

In [ ]:
def get_admin_analytics(request):
    """
    Main function called by app.py for the /admin/analytics route
    """
    with closing(get_db()) as db:
        cur = db.cursor()
        
        # Service statistics
        cur.execute("""
            SELECT s.title, s.price, COUNT(l.id) as lead_count
            FROM services s
            LEFT JOIN leads l ON l.message LIKE '%' || s.title || '%'
            WHERE s.active = 1
            GROUP BY s.id, s.title, s.price
            ORDER BY lead_count DESC
            LIMIT 10
        """)
        top_services = cur.fetchall()
        
        # Lead statistics by provider
        cur.execute("""
            SELECT 
                COALESCE(p.business_name, 'Admin') as provider_name,
                COUNT(l.id) as lead_count
            FROM leads l
            LEFT JOIN providers p ON l.provider_id = p.id
            GROUP BY l.provider_id, p.business_name
            ORDER BY lead_count DESC
        """)
        leads_by_provider = cur.fetchall()
        
        # Leads over time (last 30 days)
        cur.execute("""
            SELECT DATE(created_at) as date, COUNT(*) as count
            FROM leads
            WHERE created_at >= date('now', '-30 days')
            GROUP BY DATE(created_at)
            ORDER BY date
        """)
        daily_leads = cur.fetchall()
        
        # ZIP code performance
        cur.execute("""
            SELECT l.zip, COUNT(*) as lead_count
            FROM leads l
            WHERE l.zip IS NOT NULL AND l.zip != ''
            GROUP BY l.zip
            ORDER BY lead_count DESC
            LIMIT 10
        """)
        top_zips = cur.fetchall()
        
        # Get profile for template
        cur.execute("SELECT first_name, business_name FROM profile WHERE id = 1")
        profile_row = cur.fetchone()
        profile = {
            "first_name": profile_row["first_name"] if profile_row else "",
            "business_name": profile_row["business_name"] if profile_row else "Your Service Provider"
        }
    
    return render_template(
        "admin_analytics.html",
        top_services=top_services,
        leads_by_provider=leads_by_provider,
        daily_leads=daily_leads,
        top_zips=top_zips,
        profile=profile,
        title="Analytics Dashboard"
    )